In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/hf-seamless-m4t-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

f:\test\chatbot\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
f:\test\chatbot\env\Lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
import torch
# Áp dụng lượng tử hóa động
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [5]:
import os

output_dir = "./quantized_model"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

# Lưu mô hình lượng tử hóa
torch.save(quantized_model.state_dict(), os.path.join(output_dir, "quantized_model.pth"))
tokenizer.save_pretrained(output_dir)

('./quantized_model\\tokenizer_config.json',
 './quantized_model\\special_tokens_map.json',
 './quantized_model\\sentencepiece.bpe.model',
 './quantized_model\\added_tokens.json',
 './quantized_model\\tokenizer.json')

In [ ]:
# Tải lại mô hình lượng tử hóa
model_path = os.path.join(output_dir, "quantized_model.pth")
quantized_model.load_state_dict(torch.load(model_path))

# Tải lại tokenizer
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [6]:
from pydub import AudioSegment
import numpy as np
import io
import soundfile as sf

f:\test\chatbot\env\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
def audio_to_tensor(audio_segment):
    """Chuyển đổi AudioSegment thành tensor"""
    audio_array = np.array(audio_segment.get_array_of_samples())
    return torch.tensor(audio_array).float().unsqueeze(0)  # Thêm dimension cho batch

def tensor_to_audio(tensor):
    """Chuyển đổi tensor thành AudioSegment"""
    audio_array = tensor.squeeze().numpy()
    return AudioSegment(
        audio_array.tobytes(), 
        frame_rate=44100, 
        sample_width=audio_array.dtype.itemsize, 
        channels=1
    )

def text_to_speech(text):
    """Chuyển văn bản thành giọng nói"""
    from pyttsx3 import init
    engine = init()
    output_path = 'output.wav'
    engine.save_to_file(text, output_path)
    engine.runAndWait()
    return AudioSegment.from_file(output_path)

In [ ]:
# Tải âm thanh từ file và chuyển đổi
input_file = "path/to/input_audio.wav"
audio = AudioSegment.from_file(input_file)
audio_tensor = audio_to_tensor(audio)

# Xử lý âm thanh qua mô hình
inputs = tokenizer(audio_tensor, return_tensors="pt")
outputs = model.generate(inputs['input_ids'])
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Chuyển văn bản thành giọng nói
output_audio = text_to_speech(output_text)

# Lưu và phát âm thanh đầu ra
output_audio.export("output_speech.wav", format="wav")
output_audio.show()